# Lab 4 - Sensitive Attributes and Data Bias
Week 4 - Q3, 23/24 <br>
SEN163B: Responsible Data Analytics<br>


By <b> Sepinoud Azimi Rashti* </b> <br>
TA <b> Darsh Modi</b> <br>
Faculty of Technology, Policy, and Management (TPM)<br>

<small>*Acknowledgement: Part of this lab is loosely based on the code developed by <i><b>Agathe Balayn</b></i> and <i><b>Seda Gürses</b>. Notebook made by <b>Nadia Metoui</b></i>

***Learning Objectives***<br>
At the end of this lab, you will be able to

- Use data analytics tools to identify Sensitive attributes and proxies
- Use data analytics tools to identify Representation Bias
- Use data analytics tools to identify Historical Bias
- Understanding, Analyse and Discuss Historical, Representation and Measurement Biases

***Structure***
- Part I. Understanding the Use Case
- Part II Protected attributes, proxies
- Part III: Representation Disparities, Skew and Bias
- Part IV. Discuss Measurement Bias

<H2> </H2>

## Part I. Understanding the Use Case

<b>Use Case Description</b> In this Lab you will be exploring a use case where a Bank wants to develop an ML-based data product in the form of ADM (automate decision system) to decide whether to <b>grant</b> or <b>not to grant</b> a loan to a given applicant. To do so the Bank uses historical data containing multiple application records, characterized by information about the loan applicants (e.g., age, gender, personal situation) and information about the loan (e.g., amount, duration, purpose). Each application is labeled <i><b> good credit </b></i> if the loan had been reimbursed or <i><b>bad credit</b></i> if the loan has not been reimbursed or if there where several issues with the reimbursement.

During this lab we will not concern our selves yet with developping (coding) the data analytics product itslef.
We will rather explore the data using data analytics tools then reflect on three categories of Bias inspired by (Suresh and Guttag 2019)
- <i>Historical Bias</i>: Case Analysis, Sensitive attributes, Data skew
- <i>Representation Bias</i>: Case Analysis, Sensitive attributes, Distributions
- <i>Measurement Bias</i>: Case Analysis




<i><b>German credit dataset</b></i> contains 1000 entries with 20 attributes (7 numerical, 13 categorical) prepared by Prof. Hofmann. We will use the version hosted in the <a href="https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)">UCI Machine Learning Archive</a>.<br>

In this dataset, each entry represents a person who took a credit from a german bank. We have no information about the date/year of the data collcetion. However, for the sake of this exercise, we will assume the data was collected in 2018.  

You can download the dataset here: https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data<br>

The original data is encoded in a complex way. You can find information about the dataset and its attributes here: (https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc).<br>

We also provide you with the code to preprocess the data in more comprehensive attributes and featuers.






Analysis Steps
- Step 1: Set-up (Provided)
- Step 2: Explore and familiarize with the dataset


### Setp 1: Set-up

You first need load (usually no installation is needed) the required libraries for this part.  The main libraries are `numpy`, `pandas`, `matplotlib` and `seaborn`. We recommend using these to easily manipulate and explore the data but you are free to use any other libraries you are more familar with.

We will introduce the data visualization tool

<div class="alert alert-block alert-danger">
<b>Note:</b> Uncomment and run the next cell if you have not previously installed the libraries.
</div>


<b>Installing required libraries</b>

In [ ]:
# If you need to install any new libraries, add the code here

# install AIF 360 Toolkit
!pip install aif360


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<b>Loading required libraries</b>

In [ ]:
# Libraries for data processing and visualiztion
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Markdown, display


# we will use the Faireness Tool IBM AI Fairness 360 only to help formatting the german data set
# Please ignore this at the moment we will learn more about it in Week 6
from aif360.datasets import GermanDataset

pip install 'aif360[LawSchoolGPA]'


<b>Download the German Credit Data set</b><br>
In the following we will download the data set and https://archive.ics.uci.edu its documentation from the website and place it in the correct. At this point in the course you should be able to do this on your own. But we provide some code to guide students who still need some help. Feel free to use, modify or discard this code.

**Option 1 Google Colab:**<br>
Uncomment the following cell to download the dataset in google colab.

In [ ]:
#Download the German Credit DataSet
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
!cp german.data /usr/local/lib/python3.8/dist-packages/aif360/data/raw/german/german.data
!cp german.doc /usr/local/lib/python3.8/dist-packages/aif360/data/raw/german/german.doc

--2023-03-08 11:11:01--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79793 (78K) [application/x-httpd-php]
Saving to: ‘german.data.3’

german.data.3       100%[===================>]  77.92K  --.-KB/s    in 0.02s   

2023-03-08 11:11:02 (3.53 MB/s) - ‘german.data.3’ saved [79793/79793]

--2023-03-08 11:11:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4679 (4.6K) [application/x-httpd-php]
Saving to: ‘german.doc.3’

german.doc.3        100%[===================>]   4.57K  --.-KB/s    

**Option 2: Local environment**<br>
<div class="alert alert-block alert-danger">
<b>Note:</b> If you are working on your local environment you will have to manually add the files "german.doc" and "german.data" to the folder
"dist-packages/aif360/data/raw/german/" under your python path.<br>
(or write your script to do it deppending on the os/platform you are using)
You can find the files in the lab folder on github or download them from: <br>
<a href="https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data">https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data</a> <br>
<a href="https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc">https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc</a>
</div>

<b>Loading the dataset</b>

Here, we will load the <i><b>German credit data</b></i> in a format that is compatible with the use of the <i><b>AIF360 toolkit</b></i>. For this, you need to make use of the already implemented class of the toolkit `GermanDataset()`.

Because the data available is encoded in a complex way, we provide you with the code to preprocess it, in the function `custom_preprocessing()`. We also provide you with an example on how to actually load the data using the `GermanDataset()` class, in `preproc_and_load_data_german()`.

<small>*Note: We are not making use of AIF360 toolkit yet we will do it on week 6*</small>

In [ ]:
def preproc_and_load_data_german():
    """
    Load and pre-process german credit dataset.
    Args: -
    Returns:
        GermanDataset: An instance of GermanDataset with required pre-processing.
    """
    def custom_preprocessing(df):
        """ Custom pre-processing for German Credit Data
        """

        def group_credit_hist(x):
            if x in ['A30', 'A31', 'A32']:
                return 'None/Paid'
            elif x == 'A33':
                return 'Delay'
            elif x == 'A34':
                return 'Other'
            else:
                return 'NA'

        def group_employ(x):
            if x == 'A71':
                return 'Unemployed'
            elif x in ['A72', 'A73']:
                return '1-4 years'
            elif x in ['A74', 'A75']:
                return '4+ years'
            else:
                return 'NA'

        def group_savings(x):
            if x in ['A61', 'A62']:
                return '<500'
            elif x in ['A63', 'A64']:
                return '500+'
            elif x == 'A65':
                return 'Unknown/None'
            else:
                return 'NA'

        def group_status(x):
            if x in ['A11', 'A12']:
                return '<200'
            elif x in ['A13']:
                return '200+'
            elif x == 'A14':
                return 'None'
            else:
                return 'NA'

        def group_personal_status(x):
            if x in ['A91']:
                return 'divorced/separated'
            elif x in ['A92']:
                return 'divorced/separated/married'
            elif x in ['A93', 'A95']:
                return 'single'
            elif x in ['A94']:
                return 'married/widowed'
            else:
                return 'NA'

        def group_foreign_worker(x):
            if x in ['A201']:
                return 'yes'
            elif x in ['A202']:
                return 'no'
            else:
                return 'NA'

        #print(df)
        #print(df.shape)
        #print(df.isnull().sum().sum())
        #print(df.isin(['NA']).sum(axis=0))
        status_map = {'A91': 1.0, 'A93': 1.0, 'A94': 1.0,
                    'A92': 0.0, 'A95': 0.0}

        df['sex'] = df['personal_status'].replace(status_map)


        # group credit history, savings, and employment
        df['credit_history'] = df['credit_history'].apply(lambda x: group_credit_hist(x))
        df['savings'] = df['savings'].apply(lambda x: group_savings(x))
        df['employment'] = df['employment'].apply(lambda x: group_employ(x))
        #df['age'] = df['age'].apply(lambda x: np.float(x >= 26))
        df['status'] = df['status'].apply(lambda x: group_status(x))
        df['personal_status'] = df['personal_status'].apply(lambda x: group_personal_status(x))
        df['foreign_worker'] = df['foreign_worker'].apply(lambda x: group_foreign_worker(x))
        group_foreign_worker
        #print(df.isin(['NA']).sum(axis=0))

        # print(df)
        # uncomment if you want to save a version of the processed data
        #df.to_csv("german_credit_data_processed.csv")
        return df

    # Feature partitions
    XD_features = ['number_of_credits', 'telephone',
                     'foreign_worker', 'people_liable_for', 'skill_level', 'credit_history', 'installment_plans', 'residence_since', 'property', 'other_debtors', 'purpose', 'savings', 'employment', 'sex', 'age', 'personal_status', 'month']
    D_features = ['sex', 'age']
    Y_features = ['credit']
    X_features = list(set(XD_features)-set(D_features))
    categorical_features = ['installment_plans', 'telephone',
                     'foreign_worker', 'skill_level', 'credit_history', 'property',
                            'other_debtors', 'purpose', 'savings', 'employment', 'personal_status']

    # privileged classes
    all_privileged_classes = {"sex": [1.0],
                              "age": lambda x: x > 25}

    # protected attribute maps
    all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
                                    "age": {1.0: 'Old', 0.0: 'Young'}}

    return GermanDataset(
        label_name=Y_features[0],
        favorable_classes=[1],
        protected_attribute_names=D_features,
        privileged_classes=[all_privileged_classes[x] for x in D_features],
        instance_weights_name=None,
        categorical_features=categorical_features,
        features_to_keep=X_features+Y_features+D_features,
        features_to_drop=[],
        metadata={ 'label_maps': [{1.0: 'Good Credit', 2.0: 'Bad Credit'}],
                   'protected_attribute_maps': [all_protected_attribute_maps[x]
                                for x in D_features]},
        custom_preprocessing=custom_preprocessing)

<br><br>

### Step 2: Explore and familiarize with the dataset

<b>Q1: Analyse the dataset and answer the following:</b>
- What is the number of records?
- What is the number of attributes present with the preprocessing we provided?
- What is the list of attribute names?
- Are there missing values that could create biases?

In [ ]:
# Instanciating the German credit dataset
dataset_gcredit = preproc_and_load_data_german()

<div class="alert alert-block alert-info">
<b>Tip:</b> The documentation of "AIF360 - German credit data" dataset  can be found <a href="https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.GermanDataset.html">[HERE]</a>. </div>


Take a look at documentation of AIF360 and use existing methods to explore the dataset instance how to access the features with:<br> `dataset_gcredit.features`.

You are also free to transform the dataset into a pandas dataframe to extract the needed information.
Use <br>
    `pd_gdata = pd.DataFrame(dataset_gcredit.features, columns=dataset_gcredit.feature_names)` <br>
    to create the pandas dataframe
</div>

-

In [ ]:
### Some possible explorations ###
# Number of records:

# Number of features:

# Feature names:

# Number of missing values for each attribute ...

### Stap 3: Visualization Tools (Optional)

Last quarter one of my students introduced me to a nice data exploring tool for pandas data frames. It is called <a href="https://pypi.org/project/pandas-profiling/">[Padnas Profiling]</a>. I thought I would share it with you and let you decide what to do with it.

Remember what we said about off-the-shelf tools: When you use tools and techniques, think about their design and possible (harmful) implications.

Have Fun!



In [ ]:
# 1. Install
# Install Pandas Profiling tool
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [ ]:
# 2. Import
from pandas_profiling import ProfileReport

In [ ]:
# 3. Use
#<name of your data frame>.profile_report()
pd_gdata.profile_report()

<br><br>

## Part II Protected attributes, proxies.

### Step 1: Sensitive and Protected Attributes

<b>Q2: Identification of protected attributes</b>

a) Study the dataset and its documentation and identify which attributes that might raise unfairness concerns and should be considered protected (according to the law). Explain, in your opinion, why are these attributes protected provide exaples of bias or unfaireness for each identified attribute.

<div class="alert alert-block alert-info">
<b>Tip:</b>

Take a look at the following documents<br>
<a href="https://www.equalityhumanrights.com/en/equality-act/protected-characteristics">(1)
Protected characteristics | Equality and Human Rights Commission (UK, 2021)</a><br>
<a href="https://rm.coe.int/discrimination-artificial-intelligence-and-algorithmic-decision-making/1680925d73">(2) Discrimination, Artificial Intelligence, and Algorithmic Decision-Making (2018)</a><br>
<a href="http://ec.europa.eu/social/BlobServlet?docId=1691&langId=en&usg=AOvVaw3vI30bO3jisairH2Z7-nSl">(3) Age discrimination and European Law (2005)</a>.

Note: For your project, you have to find legal frameworks relevant to your context. Happy to help you validate your choices of frameworks.  

<div>




-

b) Study the dataset and its documentation and identify any further sensitive but "non-protect" attributes that could cause unfairness. Explain your reasoning. Provide examples of bias or unfairness related to each attribute in your context.


***Hint:*** Protected attributes are sensitive attributes protected by law. This changes from one country/region to another. Sensitive attributes are attributes that could cause historical bias and can be protected by law or not depending on these countries/regions

-

### Step 2: Identify Proxies

<b>Q3:  Identification of "spurious" proxies </b>

a) Find the proxies for the attribute "sex".

b) Find proxies for one additional protected attribut you identified in Q2-a.

c) In your opinion, why do we want to identify proxies for protected attributes in a dataset? How should you handle the proxies?

<div class="alert alert-block alert-info">
<b> Tip: </b>A proxy attribute <i>Ap</i>  is an attribute that has a similar distribution as another attribute <i>Ax</i>, so having access to the proxy attribute <i>Ap</i> provides a good knowledge of the other attribute <i>Ax</i>. For instance, in the US the zipcode is a powerful proxy for race and education, the zipcode combined with websites visited is an even more powerful proxy, names in certain languages are strong proxies for gender, etc.<br>

The simplest way to identify proxy attributes for a protected attribute <i>Ax</i> is to compute the correlation of <i>Ax</i>  with each other attributes in the dataset. The higher the corrolation (absolute value of the corrolation) the higher the likelihood an attribute is a proxy of <i>Ax</i> <br>

You can use the `corr()` function of the pandas library to compute the correlation between two attributes
</div>


   

In [ ]:
# Your code goes here

## Part III: Representation Disparities, Skew and Bias </H3>

### Step 1: Representation Disparity --> Representation Bias

<br>
<b>Q4: Is your data representative of the population (all its categories)</b>

The aim of this section is to ensure the data is representative of the population it describes. While paying particular attention to sensitive and protected attributes.

Use any combination of data visualisations, Tables, Textual "Statistics" and Textual analysis to address these guiding questions.

a) Is the dataset we are working with representative of the German population with regard to age? Add any needed code or analysis to briefly justify your answer<br>
b) Is the dataset we are working with representative of the German population with regard to gender? Add any needed code or analysis to briefly justify your answer
c) Look at the joint distribution of the attributes for sex and personal_status=divorced/separated/married. Does the dataset seem to be representative of the German population?<br>
d) Similarly, look at the distribution of foreign workers. Does the dataset seem to be representative of the German population?<br>

<div class="alert alert-block alert-info">
<b> Tip: </b> You can find demographic information from Wikipedia <a href=https://en.wikipedia.org/wiki/Demographics_of_Germany>[Here]</a>
    
Go to section <b><i>Demographic statistics</i></b> take a closer look at the most racent  <b><i>Age structure</i></b> data (it should be from 2018). Use this data to build a distribution of german population across age, then across gender and compare it to the distributions from <b><i>the German credit data</i></b> we are working with.

It is up to you how you want to justify your answer, however using visualizations will provide more points (i.e., plots and diagram)
</div>
    

In [ ]:
# write you code here
#


-

### Step 2: Outcome Skews --> Historical Bias

<b>Q5: Outcome Skews </b>

Is there a skew towards certain groups? Are these groups historically marginalised or at risk of discrimination?:<br>

Use any combination of data visualisations, Tables, Textual "Statistics" and Textual analysis to address these guiding questions.

a) Analyse the dataset, and report the numbers of males/females with bad/good credit. Do the same for "old" / " young" people in the dataset. Normalise these numbers respectively over the total number of males/females, "old"/"young" for a fair comparison. For that, you can consider having 50 individuals for each of these groups.

b) Briefly describe your findings and explain the impacts (on fairness) of using this dataset as training data (if any)

<div class="alert alert-block alert-info">
<b> Tip: </b> We provide a function for Normalised count per attribut and lable you are free to use it or implement your own method
    
`getNormalizedCount(pd_train_data, protected_attribute, label)`
</div>
    

In [ ]:
# Normalised count per attribut and lable
def getNormalizedCount(pd_train_data, protected_attribute, label):
    unnormalized_count = pd_train_data[[protected_attribute, label]].value_counts()
    counts = {}
    for attribute_value in pd_train_data[[protected_attribute]].value_counts().keys():
        counts[attribute_value[0]] = pd_train_data[[protected_attribute]].value_counts()[attribute_value]
    normalized_count = unnormalized_count[:]
    for attribute_value, credit_value in pd_train_data[[protected_attribute, label]].value_counts().keys():
        normalized_count[attribute_value, credit_value] = normalized_count[attribute_value, credit_value] * (50 / counts[attribute_value])
    return normalized_count

# add the credit labels to the data set.
pd_gdata["credit"] = dataset_gcredit.labels

In [ ]:
### YOUR ANSWER HERE ###
# ADD code here to print the AGE-CREDIT distribution


# ADD code here to print the SEX-CREDIT distribution


# ADD code here to visualise the results for both you can use stacked bar plots from pandas toolkit
#<your dataframe>.size().unstack().plot(kind='bar', stacked=True)


-

## Part IV. Discuss Measurement Bias.

Measurement Biases occur when choosing, collecting, or computing inadequate features and labels to use in a prediction problem. Features and labels are the concrete measurements chosen to approximate some construct (an idea or concept) that is not directly encoded or observable.
</br></br>

<b>Q6: Reflect on your featuers and lables</b> </br>
There is no "simple" answer to this question, nor a "technical" quick fix. Your reflection and analysis should be contextual and responsible. Use outputs from Lecture 1 and the Responsible Data Guiding Principles from (D’Ignazio & Klein, 2016) to carry out your analysis and identify measurement bias in your data project. In the following, we provide a set of guiding questions that could help you articulate your thoughts and discussion for this type of bias.

***Sample of Guiding questions:***

a) Are you using inclusive non-harmful featuers and lables?

b) Is your dataset making any harmful or discriminatory abstractions.

c) Are you using the correct "proxy"/"lable" to predict the correct "concept"

***Important Note***
The german data set is a processed and simplified dataset. It is not obvious what assumptions were made to clean and label the dataset there, for it is hard to identify Measurement Biases in this dataset. It does not mean it is bias-free. On the contrary, this should warn you about using pre-processed, pre-packaged datasets. They are loaded with biases you can not identify and asses


## End of the Lab, Well Done!